<a href="https://colab.research.google.com/github/honggi82/computer_vision/blob/main/yolov12_webcam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Colab이 아닌 일반 PC환경에서는 아래와 같이 source='0'으로 설정하여 웹캠을 인식할 수 있음

from ultralytics import YOLO

# 1. 모델 로드
model = YOLO('yolo11n.pt')

# 2. 웹캠 실시간 추론
# source=0 : 기본 웹캠
# show=True : 화면에 실시간으로 결과창 띄우기
results = model.predict(source='0', show=True, stream=True)

# 3. 실시간 처리 루프 (stream=True일 때 필요)
for result in results:
    pass  # 화면(show=True)에서 이미 보여주므로 여기선 별도 코드 불필요

In [ ]:
!pip install ultralytics

In [ ]:
import cv2
import numpy as np
import PIL.Image
import io
import base64
from ultralytics import YOLO
from IPython.display import display, Javascript
from google.colab.output import eval_js

In [ ]:
# ==========================================
# 1. 유틸리티 함수 (이미지 변환 등)
# ==========================================
def js_to_image(js_reply):
    """JS 객체를 OpenCV 이미지로 변환"""
    image_bytes = base64.b64decode(js_reply.split(',')[1])
    jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
    img = cv2.imdecode(jpg_as_np, flags=1)
    return img

def bbox_to_bytes(bbox_array):
    """OpenCV 사각형 이미지를 base64 문자열로 변환 (오버레이용)"""
    bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
    iobuf = io.BytesIO()
    bbox_PIL.save(iobuf, format='png')
    bbox_bytes = 'data:image/png;base64,{}'.format((str(base64.b64encode(iobuf.getvalue()), 'utf-8')))
    return bbox_bytes

# ==========================================
# 2. JavaScript 스트리밍 함수 정의
# ==========================================
def video_stream():
    js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;

    var pendingResolve = null;
    var shutdown = false;

    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }

    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }

    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);

      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);

      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);

      const instruction = document.createElement('div');
      instruction.innerHTML =
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };

      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640;
      captureCanvas.height = 480;
      window.requestAnimationFrame(onAnimationFrame);

      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();

      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }

      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }

      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;

      return {'create': preShow - preCreate,
              'show': preCapture - preShow,
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

    display(js)

def video_frame(label, bbox):
    data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
    return data

In [ ]:
# ==========================================
# 3. 메인 실행 루프 (Ultralytics 적용)
# ==========================================

# YOLO 모델 로드
model = YOLO('yolo12n.pt')

# 웹캠 스트림 시작
video_stream()
label_html = 'Capturing...'
bbox = ''

# 색상 랜덤 생성 (시각화용)
colors = np.random.uniform(0, 255, size=(100, 3))

print("웹캠 영상 분석 시작 (화면을 클릭하면 종료됩니다)")

while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break

    # 1. JS에서 받은 이미지를 OpenCV 포맷으로 변환
    frame = js_to_image(js_reply["img"])

    # 2. 투명 오버레이 레이어 생성 (이 위에 박스를 그림)
    # JS 코드에서 width=640, height=480으로 고정되어 있음
    bbox_array = np.zeros([480, 640, 4], dtype=np.uint8)

    # 3. Ultralytics YOLO 추론 수행
    # verbose=False로 설정하여 로그 출력 최소화
    results = model(frame, verbose=False)

    # 4. 결과 처리 및 그리기
    for result in results:
        # 감지된 각 객체(box)에 대해 반복
        for box in result.boxes:
            # 좌표 추출 (x1, y1, x2, y2)
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)

            # 신뢰도(Confidence)와 클래스 ID 추출
            conf = float(box.conf[0].cpu().numpy())
            cls = int(box.cls[0].cpu().numpy())

            # 클래스 이름 가져오기
            label_name = result.names[cls]
            label_text = f"{label_name} [{conf:.2f}]"

            # 색상 지정
            color = colors[cls % len(colors)]

            # 사각형 그리기 (OpenCV)
            cv2.rectangle(bbox_array, (x1, y1), (x2, y2), color, 2)

            # 텍스트 그리기
            cv2.putText(bbox_array, label_text, (x1, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # 5. 그려진 오버레이 레이어의 투명도 처리 (박스 있는 곳만 불투명하게 255)
    bbox_array[:,:,3] = (bbox_array.max(axis=2) > 0).astype(int) * 255

    # 6. 오버레이 이미지를 다시 바이트로 변환하여 JS로 전송
    bbox_bytes = bbox_to_bytes(bbox_array)

    # 다음 프레임을 위해 bbox 변수 업데이트
    bbox = bbox_bytes

<IPython.core.display.Javascript object>

웹캠 영상 분석 시작 (화면을 클릭하면 종료됩니다)


/tmp/ipython-input-1768543356.py:13: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')


KeyboardInterrupt: 